# Análise de Dados de Filmes

Este notebook contém várias análises sobre dados de filmes, explorando diferentes aspectos como orçamento, receita, equipe, gêneros e muito mais.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Carregando o dataset
script_dir = os.path.dirname(os.path.abspath('__file__'))
DATASET_PATH = os.path.join(script_dir, 'movies.csv')

try:
    df = pd.read_csv(DATASET_PATH)
    print("Dataset carregado com sucesso!")
    print(f"Número de filmes: {len(df)}")
    print("\nPrimeiras linhas do dataset:")
    display(df.head())
except FileNotFoundError:
    print("Erro: Arquivo movies.csv não encontrado")

## 1. Análise de Orçamento e Receita

Vamos analisar a relação entre o orçamento dos filmes e sua receita, calculando também o ROI (Return on Investment).

In [ ]:
def analise_orcamento_receita(df):
    # Calculando o ROI
    df['ROI'] = (df['revenue'] - df['budget']) / df['budget']

    # Criando o gráfico de dispersão
    plt.figure(figsize=(10, 6))
    plt.scatter(df['budget'], df['revenue'], alpha=0.5)
    plt.xlabel('Orçamento')
    plt.ylabel('Receita')
    plt.title('Relação entre Orçamento e Receita')
    plt.show()

    # Calculando a correlação
    correlacao = df['budget'].corr(df['revenue'])
    print(f"Correlação entre orçamento e receita: {correlacao:.2f}")
    
    # Exibindo estatísticas do ROI
    print(f"\nEstatísticas do ROI:")
    print(df['ROI'].describe())

analise_orcamento_receita(df)

## 2. Análise de Produtoras

Vamos investigar como o número de produtoras envolvidas se correlaciona com o orçamento e a bilheteria dos filmes.

In [ ]:
def analise_produtoras(df):
    print("Análise de produtoras em desenvolvimento...")
    # Implementação futura

analise_produtoras(df)

## 3. Análise de Diretores e Atores

Vamos identificar quais diretores trabalham frequentemente com os mesmos atores em múltiplos filmes.

In [ ]:
def analise_diretores_atores(df):
    print("Análise de diretores e atores em desenvolvimento...")
    # Implementação futura

analise_diretores_atores(df)

## 4. Análise de Duração e Popularidade

Vamos investigar se existe alguma relação entre a duração do filme e sua popularidade ou avaliação.

In [ ]:
def analise_duracao_popularidade(df):
    print("Análise de duração e popularidade em desenvolvimento...")
    # Implementação futura

analise_duracao_popularidade(df)

## 5. Análise de Diversidade de Gênero

Vamos analisar a diversidade de gênero nas equipes de produção dos filmes.

In [ ]:
def analise_diversidade_genero(df):
    print("Análise de diversidade de gênero em desenvolvimento...")
    # Implementação futura

analise_diversidade_genero(df)

## 6. Análise de Gêneros e Orçamento

Vamos investigar quais gêneros têm maiores orçamentos e se isso resulta em maior receita.

In [ ]:
def analise_generos_orcamento(df):
    print("Análise de gêneros e orçamento em desenvolvimento...")
    # Implementação futura

analise_generos_orcamento(df)

## 7. Análise de Tamanho da Equipe

Vamos analisar como o tamanho da equipe se relaciona com o orçamento e o sucesso dos filmes.

In [ ]:
def analise_equipe_orcamento(df):
    print("Análise de equipe e orçamento em desenvolvimento...")
    # Implementação futura

analise_equipe_orcamento(df)

## 8. Análise de Distribuição Geográfica

Vamos analisar a distribuição geográfica das produções e sua relação com orçamento e receita.

In [ ]:
def analise_distribuicao_geografica(df):
    print("Análise de distribuição geográfica em desenvolvimento...")
    # Implementação futura

analise_distribuicao_geografica(df)

## 9. Análise de Data de Lançamento

Vamos investigar se existe correlação entre a data de lançamento e o desempenho de bilheteria.

In [ ]:
def analise_data_bilheteria(df):
    print("Análise de data de lançamento em desenvolvimento...")
    # Implementação futura

analise_data_bilheteria(df)

## 10. Análise Temporal de Orçamento e Receita

Vamos analisar como o orçamento e a receita dos filmes mudaram ao longo do tempo.

In [ ]:
def analise_orcamento_tempo(df):
    print("Análise temporal de orçamento e receita em desenvolvimento...")
    # Implementação futura

analise_orcamento_tempo(df)